In [0]:
# Importando as functions

from pyspark.sql.functions import sum as sum_, col

In [0]:
# Lendo DataFrame
pedidos_df = spark.table("ecommerce.silver.pedidos")
clientes_df = spark.table("ecommerce.silver.clientes")
itens_pedido_df = spark.table("ecommerce.silver.itens_pedido")
produtos_df = spark.table("ecommerce.silver.produtos")
categorias_df = spark.table("ecommerce.silver.categorias")
pagamentos_df = spark.table("ecommerce.silver.pagamentos")

In [0]:
# Alias das tabelas

i = itens_pedido_df.alias("i")
p = pedidos_df.alias("p")
c = clientes_df.alias("c")
pr = produtos_df.alias("pr")
cat = categorias_df.alias("cat")
pg = pagamentos_df.alias("pg")
ia = itens_agg_df.alias("ia")

pedidos_completos_df = (
    i
    
    # Pedido
    .join(p, "pedido_id", "left")
    
    # Cliente
    .join(c, "cliente_id", "left")
    
    # Produto
    .join(pr, "produto_id", "left")
    
    # Categoria
    .join(cat, "categoria_id", "left")
    
    # Pagamento
    .join(pg, "pedido_id", "left")
    
    # Total itens do pedido
    .join(ia, "pedido_id", "left")
    
    .select(

        # Cliente
        col("c.nome").alias("cliente_nome"),
        col("c.cidade"),
        col("c.estado"),
        col("c.regiao"),

        # Produto
        col("pr.nome").alias("produto_nome"),
        col("cat.nome").alias("categoria_nome"),
        
        # Pedido
        col("p.data_pedido"),
        col("p.valor_liquido"),
        
        # Pagamento
       col("pg.status").alias("status_pagamento"),
       col("pg.metodo").alias("metodo_pagamento"),

        
        # Métrica
        col("i.quantidade"),
        col("ia.quantidade_itens")
    )
)

In [0]:
#Salvando DataFrame na camada silver
pedidos_completos_df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("ecommerce.silver.pedidos_completos")